In [1]:
import glob
import pandas as pd
import re
import nilearn as nil
from nilearn import *
from nilearn import image
from os.path import basename
import numpy as np
import yaml
from yaml.loader import SafeLoader
from socket import gethostname

from get_all_series import get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed
from get_all_series import get_all_subj_df

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
gethostname()

'n137'

In [3]:
# Open the file and load the file
with open('config.yml') as f:
    all_yaml = yaml.load(f, Loader=SafeLoader)
    if gethostname() in all_yaml.keys():
        config = all_yaml[gethostname()]
    else:
        config = all_yaml['default']

In [4]:
config

{'dropbox_data_dir': '/gpfs/projects/sanlab/bsmith16/data',
 'fmriprep_dir': '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/',
 'nii_raw_path': 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii',
 'mask_location': '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/masks/'}

get all series

get masks

iterate through series

iterate through masks

produce the vector for this mask/series

for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;
this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
for Go trials it's harder to specify.

for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
these can be concatenated across trials, runs, and subjects

then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.


In [5]:
# #need to replace these with references to the config yml
# dropbox_data_dir = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'
# fmriprep_dir = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/'
# nii_raw_path = 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii'
# mask_location = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/striatum/'
dropbox_data_dir = config['dropbox_data_dir']
fmriprep_dir = config['fmriprep_dir']
nii_raw_path = config['nii_raw_path']
mask_location = config['mask_location'] + 'striatum/'

get all series

In [6]:
glob_path = fmriprep_dir + nii_raw_path
print("looking up " + glob_path)
nii_raw_files = glob.glob(glob_path)

looking up /gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii


In [7]:
print(nii_raw_files)

['/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV033/ses-wave1/func/s6_sub-DEV033_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV079/ses-wave1/func/s6_sub-DEV079_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV017/ses-wave1/func/s6_sub-DEV017_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV131/ses-wave1/func/s6_sub-DEV131_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV122/ses-wave1/func/s6_sub-DEV122_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep/sub-DEV138/ses-wave1/func/s6_sub-DEV138_ses-wave1_task-SST_

get masks

In [8]:
mask_location

'/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/masks/striatum/'

In [9]:
mask_paths = glob.glob(mask_location+"*.nii.gz")
mask_labels = [re.match(".*/(.*)\.nii\.gz",mp)[1] for mp in mask_paths]

mask_df = pd.DataFrame({
    'mask_label':mask_labels,
    'mask_path': mask_paths
})

In [10]:
mask_df

,mask_label,mask_path
0,harvardoxford-subcortical_prob_Left Caudate,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...
1,harvardoxford-subcortical_prob_Right Accumbens,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...
2,harvardoxford-subcortical_prob_Right Putamen,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...
3,harvardoxford-subcortical_prob_Left Putamen,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...
4,harvardoxford-subcortical_prob_Right Caudate,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...
5,harvardoxford-subcortical_prob_Left Accumbens,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...


iterate through series

In [11]:
#mark the number of seconds each scan takes.
#TR = 2.0
roi_data = get_roi_data(nii_raw_files,mask_df)



DEV033, 1


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/image/image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(


harvardoxford-subcortical_prob_Left Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
DEV079, 1


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/image/image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(


harvardoxford-subcortical_prob_Left Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
DEV017, 1


...

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/image/image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(


harvardoxford-subcortical_prob_Left Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
DEV123, 1


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/image/image.py:1106: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  data = signal.clean(


harvardoxford-subcortical_prob_Left Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Putamen
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Caudate
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Accumbens
producing the vector for this mask/series...
created an activity vector for this mask with the following length: 212
DEV168, 1


KeyboardInterrupt: 

In [ ]:
roi_data is None

In [ ]:
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        print(wave)
        print(roi_data[s][wave].shape)
        print(roi_data[s][wave].columns)

In [ ]:
sst_all_behavioral_data = pd.read_csv(dropbox_data_dir + "/sst_behavioral_data_all.csv")



for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

In [ ]:


sst_all_behavioral_data = get_behavioral_data_with_moment_trial_type_revealed(sst_all_behavioral_data)



In [ ]:
sst_all_behavioral_data['class_type_reveal_onset'] = sst_all_behavioral_data.class_type_reveal + sst_all_behavioral_data.onset
sst_all_behavioral_data.loc[sst_all_behavioral_data.go_no_go_condition_label=='Cue','class_type_reveal_onset']=None

In [ ]:
roi_data

Now we need a list of the TRs by their time of appearance; that way we can mark which ones are relevant to the ROIs for each trial.

For now, let's assume that the timing of the TRs matches the timing in the matlab files, but this MUST be checked.

In [ ]:
# run_length = 207
# rt_time_list = [x*2.0 for x in range(run_length)]

In [ ]:
import pickle

with open(dropbox_data_dir + '/roi_data_raw.pkl', 'wb') as handle:
    pickle.dump(roi_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# import pickle

# with open(dropbox_data_dir + '/roi_data_raw.pkl', 'rb') as handle:
#     roi_data = pickle.load(handle)

In [ ]:
dropbox_data_dir + '/roi_data_raw.pkl'

these are going to be generated dynamically based on the run length

In [ ]:
#from get_all_series import get_all_subj_df
all_subj_df = get_all_subj_df(roi_data, sst_all_behavioral_data)

In [ ]:
all_subj_df.groupby(['trial_n','subid','wave']).offset.count()

In [ ]:
all_subj_df.to_csv(dropbox_data_dir + '/SST_roi_by_time_point.csv')

We should have up to 420 or so.

In [ ]:
sst_all_behavioral_data.loc[sst_all_behavioral_data.subid=='DEV293'][['trial_n','onset','class_type_reveal_onset','condition']]

In [ ]:

# for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
# these can be concatenated across trials, runs, and subjects

# then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

# potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.
